# Time Series Polling
This short notebook shows how to stream RHEED time series updates using the polling helpers in `atomicds.timeseries.polling`.

### Install package

In [ ]:
#!pip install atomicds

### API Client Setup

In [ ]:
from atomicds.client import Client
from atomicds.timeseries.polling import iter_poll, aiter_poll, start_polling_thread, start_polling_task

api_key = "YOUR_API_KEY_HERE"
client = Client(api_key=api_key)

# Use a data ID that has RHEED time series data available to your account.
data_id = "YOUR_TIME_SERIES_DATA_ID"

### Sync polling loop
Iterate over `iter_poll` to fetch the latest rows on a fixed cadence.
This example polls every five seconds, keeps the latest ten rows, avoids duplicate results based on the newest timestamp, and stops after three successful polls.

In [ ]:
def latest_timestamp(df):
    if df.empty or "timestamp" not in df.columns:
        return None
    return df.iloc[-1]["timestamp"]

for idx, result in enumerate(
    iter_poll(
        client,
        data_id=data_id,
        interval=5.0,
        last_n=10,
        distinct_by=latest_timestamp,
        max_polls=3,
    ),
    start=1,
):
    print(f"Poll {idx}: latest timestamp -> {latest_timestamp(result)}")
    print(result.tail())

### Background thread helper
`start_polling_thread` runs the same sync poller in a daemon thread and streams each result to a callback.
Use the returned event to stop early if needed.

In [ ]:
collected = []


def on_result(result):
    print(f"Thread received {len(result)} rows")
    collected.append(result)

stop_event = start_polling_thread(
    client,
    data_id=data_id,
    interval=10.0,
    last_n=10,
    max_polls=5,
    distinct_by=latest_timestamp,
    on_result=on_result,
)

# Call `stop_event.set()` if you need to halt polling before `max_polls` is reached.

### Async polling loop
`aiter_poll` yields the same results without blocking the event loop.
Top-level `await` works in Jupyter; otherwise wrap it in `asyncio.run`.

In [ ]:
import asyncio

async def stream_updates():
    async for result in aiter_poll(
        client,
        data_id=data_id,
        interval=5.0,
        last_n=10,
        distinct_by=latest_timestamp,
        max_polls=3,
    ):
        print(f"Async poll received {len(result)} rows")
        print(result.tail())

await stream_updates()

### Async background task
`start_polling_task` creates an `asyncio.Task` that forwards each poll to a handler.
The handler can be sync or async; async handlers will be awaited before the next poll.

In [ ]:
async def handle_async(result):
    print(f"Task handler received {len(result)} rows")

polling_task = start_polling_task(
    client,
    data_id=data_id,
    interval=5.0,
    last_n=5,
    max_polls=3,
    distinct_by=latest_timestamp,
    on_result=handle_async,
)

await polling_task